#1. Setup Mock Database

In [ ]:
# Import the sqlite3 library
# sqlite3 is a built-in Python library that allows us to create and interact with SQLite databases.
# SQLite is lightweight, serverless, and stores data in a single file, making it perfect for testing and learning.
import sqlite3

# Create an in-memory database (it exists only while the program runs).
# ":memory:" means the database is stored in RAM, not on disk.
# This is useful for labs, as everything is temporary and resets each time you run the code.
conn = sqlite3.connect(":memory:")

# Create a cursor object which is used to execute SQL commands (queries).
cursor = conn.cursor()

# Create a mock "users" table in our database.
# The table has three columns:
# - id (auto-incremented primary key, unique for each row)
# - username (text field to store the user’s name)
# - password (text field to store the user’s password)
cursor.execute("""
CREATE TABLE users (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    username TEXT,
    password TEXT
)
""")

# Prepare some sample user data.
# Each entry is a tuple: (username, password)
users = [
    ("admin", "admin123"),
    ("john", "john123"),
    ("alice", "alice123")
]

# Insert the sample user data into the "users" table.
# The question marks (?) are placeholders for values (to prevent injection here).
# executemany() allows inserting multiple rows at once.
cursor.executemany("INSERT INTO users (username, password) VALUES (?, ?)", users)

# Save (commit) the changes to the database.
# In this case, it finalizes the insertion of our sample users.
conn.commit()

# Confirmation message to show the database and users are ready.
print(" Database setup complete with sample users!")


 Database setup complete with sample users!


#Display All Users in the Database

In [ ]:
# Fetch all records from the users table
cursor.execute("SELECT * FROM users")
rows = cursor.fetchall()

# Print results in a table-like format
print(" Users Table:")
print("ID | Username | Password")
print("-" * 30)
for row in rows:
    print(f"{row[0]}  | {row[1]}      | {row[2]}")


 Users Table:
ID | Username | Password
------------------------------
1  | admin      | admin123
2  | john      | john123
3  | alice      | alice123


Use Pandas for a Cleaner Table

In [ ]:
import pandas as pd

# Load table into a pandas DataFrame for a clean display
cursor.execute("SELECT * FROM users")
df = pd.DataFrame(cursor.fetchall(), columns=["ID", "Username", "Password"])
df


,ID,Username,Password
0,1,admin,admin123
1,2,john,john123
2,3,alice,alice123


#2. Vulnerable Login Function (Unsafe)

In [ ]:
def vulnerable_login(username, password):
    """
    This function simulates a vulnerable login mechanism.
    It is intentionally written in an unsafe way to demonstrate SQL Injection.
    """

    #  Vulnerable part: User inputs are directly concatenated into the SQL query string.
    # If the input contains SQL code (e.g., ' OR '1'='1), it will modify the query.
    query = f"SELECT * FROM users WHERE username = '{username}' AND password = '{password}'"

    # Display the exact query being executed.
    # This helps students see how their input changes the query.
    print("Executing Query:", query)

    # Execute the constructed SQL query.
    # Since the query string may contain injected code, it can change the intended logic.
    cursor.execute(query)

    # Fetch all matching rows from the database.
    result = cursor.fetchall()

    # If any rows are returned, login is considered "successful".
    # Otherwise, login fails.
    if result:
        return "Login Successful!"
    else:
        return "Login Failed!"



#Testing the Vulnerable Login Function

In [ ]:
# 1. Normal Login (Correct Password)
print(vulnerable_login("john", "john123"))

Executing Query: SELECT * FROM users WHERE username = 'john' AND password = 'john123'
Login Successful!


In [ ]:
# 2. Normal Login (Wrong Password)
print(vulnerable_login("john", "wrongpass"))

Executing Query: SELECT * FROM users WHERE username = 'john' AND password = 'wrongpass'
Login Failed!


In [ ]:
# 3. SQL Injection via Username
print(vulnerable_login("' OR '1'='1' --", "anything"))

Executing Query: SELECT * FROM users WHERE username = '' OR '1'='1' --' AND password = 'anything'
Login Successful!


In [ ]:
# 4. SQL Injection via Password
print(vulnerable_login("john", "' OR '1'='1' --"))

Executing Query: SELECT * FROM users WHERE username = 'john' AND password = '' OR '1'='1' --'
Login Successful!


In [ ]:
print(vulnerable_login("john", "john123"))   # Correct password
print(vulnerable_login("john", "wrongpass")) # Wrong password


Executing Query: SELECT * FROM users WHERE username = 'john' AND password = 'john123'
Login Successful!
Executing Query: SELECT * FROM users WHERE username = 'john' AND password = 'wrongpass'
Login Failed!


In [ ]:
# Injecting into the password field
print(vulnerable_login("john", "' OR '1'='1"))

# Injecting into the username field
print(vulnerable_login("' OR '1'='1", "anything"))


Executing Query: SELECT * FROM users WHERE username = 'john' AND password = '' OR '1'='1'
Login Successful!
Executing Query: SELECT * FROM users WHERE username = '' OR '1'='1' AND password = 'anything'
Login Failed!


In [ ]:
def safe_login(username, password):
    query = "SELECT * FROM users WHERE username = ? AND password = ?"
    cursor.execute(query, (username, password))
    result = cursor.fetchall()
    if result:
        return " Login Successful (SAFE)!"
    else:
        return " Login Failed (SAFE)!"

print(safe_login("john", "' OR '1'='1"))


 Login Failed (SAFE)!
